In [ ]:
!python -m pip install matplotlib numpy

In [ ]:
from math import 
def f(x: float) -> float:
    return 3.0 * sin(x * 3.0) + 5.0 * cos()

start, stop = -7.0, 17.0
x = np.linspace(start, stop, 2 ** 10 + 1)
f_vectorize = np.vectorize(f)
y = f_vectorize(x)